In [ ]:
import numpy as np

def decision_module(ball_trajectory, velocity_vectors, object_metadata,
                    orientation_vectors, audio_waveform, leg_position_data):
    """
    Bat's Edge Detection Module for Decision Review System (DRS).
    Determines whether a ball has made contact with the bat and routes data accordingly.
    """

    def compute_proximity(ball_pos, bat_bbox):
        """
        Estimate proximity using the bat's bounding box center and current ball position.
        """
        x1, y1, x2, y2 = bat_bbox
        bat_center = np.array([(x1 + x2) / 2, (y1 + y2) / 2, 0])
        return np.linalg.norm(np.array(ball_pos[:2]) - bat_center[:2])

    def detect_audio_spike(audio_waveform, threshold=0.75):
        """
        Detect sharp spike in audio signal representing a potential edge sound.
        """
        max_val = max(audio_waveform)
        return max_val >= threshold

    def detect_velocity_anomaly(velocity_vectors):
        """
        Checks for sudden change in velocity (deflection detection).
        """
        if len(velocity_vectors) < 2:
            return False
        v1 = np.array(velocity_vectors[-2])
        v2 = np.array(velocity_vectors[-1])
        delta_v = np.linalg.norm(v2 - v1)
        return delta_v > 2.0


    bat_info = object_metadata.get("bat", {})
    bat_bbox = bat_info.get("bbox", [])
    bat_confidence = bat_info.get("confidence", 0)

    if not bat_bbox or bat_confidence < 0.8:

        decision_flag = (False, None)
        return {
            "module": "Trajectory Analysis Module",
            "original_trajectory": ball_trajectory,
            "leg_position_data": leg_position_data,
            "decision_flag": decision_flag
        }


    ball_pos = ball_trajectory[-1]
    proximity = compute_proximity(ball_pos, bat_bbox)


    edge_detected = (
        proximity < 15.0 and
        detect_audio_spike(audio_waveform) and
        detect_velocity_anomaly(velocity_vectors)
    )

    if edge_detected:
        contact_point = ball_trajectory[-1]
        updated_velocity = velocity_vectors[-1]
        decision_flag = (True, contact_point)

        return {
            "module": "Stream Analysis and Overlay Module",
            "updated_velocity": updated_velocity,
            "contact_point": contact_point,
            "decision_flag": decision_flag
        }

    else:
        decision_flag = (False, None)
        return {
            "module": "Trajectory Analysis Module",
            "original_trajectory": ball_trajectory,
            "leg_position_data": leg_position_data,
            "decision_flag": decision_flag
        }